# Tic-Tac-Toe Library Tutorial

This notebook shows how to use the TicTacToe library: start a game, make moves, detect wins/draws, render the board, and call the built-in AI.

What's explained:
- Create a game (newGame) and read game.board, game.current, and game.state.
- Validate moves and handle errors (occupied cells, out-of-bounds, game over).
- Render a simple CLI board view from the read-only board data.
- Use AI players: AIPlayer.getRandomMove and AIPlayer.computeBestMove.

Design goals:
- Small, UI-agnostic game engine (no UI dependencies).
- Clear public API for game state and moves.
- Built-in AI (Minimax) suitable for 3x3 boards.

## 0. Setup
Open this Gradle project in IntelliJ IDEA. After Gradle sync, run each cell. If imports fail, build the project first.

In [19]:
// Import the library API and AI:
import org.jetbrains.kotlinx.tictactoe.*
println("TicTacToe library loaded.")

TicTacToe library loaded.


## 1. TicTacToe library overview

Types:
- Mark: enum { X, O }
- GameState: InProgress | Won(winner: Mark) | Draw
- TicTacToe: interface exposing board, current, state, and play(row, col)
- newGame(): factory returning a TicTacToe implementation

Rules:
- 3x3 board, rows/cols in 0..2
- X moves first, then players alternate
- IllegalArgumentException for out-of-bounds or occupied cells
- IllegalStateException if trying to play after game over

## 2. Create a game and inspect initial state


In [20]:
val g = newGame()

println("Initial board:")
println(g.board) // 3x3 array of nulls

Initial board:
[[null, null, null], [null, null, null], [null, null, null]]


Displaying the board in a UI is up to you. The `board` property is a 2D array of nullable `Mark` values. Here's a simple text rendering in console:

In [21]:
fun render(game: TicTacToe) {
    for (row in game.board) {
        for (cell in row) {
            when (cell) {
                Mark.X -> print(" X ")
                Mark.O -> print(" O ")
                null -> print(" . ")
            }
        }
        println()
    }
}
render(g)

 .  .  . 
 .  .  . 
 .  .  . 


## 3. Playing and handling invalid moves
Out-of-bounds, already-occupied cells, or moves after game over throw exceptions.

In [22]:
g.play(0, 0) // First move by X

try {
    g.play(0, 0) // already occupied
} catch (e: IllegalArgumentException) {
    println("Expected: ${e.message}")
}
try {
    g.play(3, 0) // out of bounds
} catch (e: IllegalArgumentException) {
    println("Expected: ${e.message}")
}

Expected: Cell (0,0) is already occupied
Expected: Row and column must be in 0..2


## 4. Game States: Win, Draw, InProgress
The game state updates automatically after each valid move. You can check the state and current player at any time via g.state and g.current

In [23]:
val win = newGame()
win.play(0, 0) // X
win.play(1, 0) // O
render(win)
println("State: ${win.state}, Current Player: ${win.current}")
win.play(0, 1) // X
win.play(1, 1) // O
win.play(0, 2) // X wins row 0
render(win)
println("State: ${win.state}")

val draw = newGame()
draw.play(0,0); draw.play(0,1); draw.play(0,2)
draw.play(1,1); draw.play(1,0); draw.play(1,2)
draw.play(2,1); draw.play(2,0); draw.play(2,2)
render(draw)
println("State: ${draw.state}")

 X  .  . 
 O  .  . 
 .  .  . 
State: InProgress, Current Player: X
 X  X  X 
 O  O  . 
 .  .  . 
State: Won(winner=X)
 X  O  X 
 X  O  O 
 O  X  X 
State: Draw


## Using the built-in AI

### Random moves
The AI can pick random valid moves.

In [24]:
val randomGame = newGame()
randomGame.play(0, 0) // X
randomGame.play(1, 1) // O
randomGame.play(0, 1) // X

render(randomGame)

val random = AIPlayer.getRandomMove(randomGame)

println("AI (for O) suggests random move: $random")
if (random != null) {
    randomGame.play(random.first, random.second)
}
render(randomGame)
println("State: ${randomGame.state}, Current: ${randomGame.current}")


 X  X  . 
 .  O  . 
 .  .  . 
AI (for O) suggests random move: (0, 2)
 X  X  O 
 .  O  . 
 .  .  . 
State: InProgress, Current: X


### Minimax Algorithm
AI selects the best move for the current player. It prefers faster wins and slower losses.

In [25]:
val minmaxGame = newGame()
minmaxGame.play(0, 0) // X
minmaxGame.play(1, 1) // O
minmaxGame.play(0, 1) // X

render(minmaxGame)

val best = AIPlayer.computeBestMove(minmaxGame)

println("AI (for O) should block or win if possible. Suggested move: $best")

if (best != null) {
    minmaxGame.play(best.first, best.second)
}
render(minmaxGame)
println("State: ${minmaxGame.state}, Current: ${minmaxGame.current}")

 X  X  . 
 .  O  . 
 .  .  . 
AI (for O) should block or win if possible. Suggested move: (0, 2)
 X  X  O 
 .  O  . 
 .  .  . 
State: InProgress, Current: X


### Summary of integration steps
- Map user input (click/tap/keystroke) to a (row, col) pair and call `game.play(row, col)`.
- Refresh your view from `game.board` after every move; don't mutate the board directly.
- Show the current player with `game.current`.
- Detect game over via `game.state`; disable further input and show a win/draw message.
- Coordinates are 0..2 (top-left is 0,0). Consider showing axis labels in your UI.
- For AI turns, compute a move (`AIPlayer.computeBestMove` or `getRandomMove`) and then call `game.play`.

## Notes on design and possible extensions
- Read-only board: exposed as `List<List<Mark?>>`; all validation happens inside `play`.
- Exceptions: `IllegalArgumentException` for out-of-bounds or occupied cells; `IllegalStateException` if playing after game over.
- AI: Minimax is fast enough for 3x3 and prefers faster wins/slower losses; `getRandomMove` is non-deterministic.
- Extensions: custom renderers (CLI/desktop/web), input methods, alternative AIs, different scoring.
- Threading: the engine is designed for single-threaded turn-taking; synchronize access if you update from multiple threads.
- Persistence: to save/restore, store a 3x3 grid of "X"/"O"/null or a move history; rebuild via `newGame()` + replays.
